In [33]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

# Getting real estate listings


## Zumber

## Studios

In [34]:
URL = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/studios"
requestURL = requests.get(URL)
content = requestURL.content
soup = BeautifulSoup(content, "html.parser")
rent = [x.text for x in soup.find_all("div", class_="css-uzc059")]
bath = [x.text for x in soup.find_all("div", class_="css-1iq6kk8")]


dict = {'Monthly': rent, 'Number of baths': bath}
df= pd.DataFrame(dict)
df['Monthly'] = df['Monthly'].str.replace('\D+', '')

In [40]:
df

,Monthly,Number of baths
0,1900,Studio • 1 Bath
1,2369,Studio • 1 Bath
2,2268,Studio • 1 Bath
3,2570,Studio • 1 Bath
4,2650,Studio • 1 Bath
5,2450,Studio • 1 Bath
6,2495,Studio • 1 Bath
7,3500,Studio • 1 Bath
8,2700,Studio • 1 Bath
9,2375,Studio • 1 Bath


## 1 bed

In [49]:
URL = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/1-beds"
requestURL = requests.get(URL)
content = requestURL.content
soup = BeautifulSoup(content, "html.parser")
rent = [x.text for x in soup.find_all("div", class_="css-uzc059")]
bath = [x.text for x in soup.find_all("div", class_="css-1iq6kk8")]


dict = {'Monthly': rent, 'Number of baths': bath}
dfOneBed= pd.DataFrame(dict)
dfOneBed['Monthly'] = dfOneBed['Monthly'].str.replace('\D+', '')

C:\Users\Jimmy\AppData\Local\Temp/ipykernel_9092/1776082707.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  dfOneBed['Monthly'] = dfOneBed['Monthly'].str.replace('\D+', '')


In [50]:
dfOneBed

,Monthly,Number of baths
0,3875,1 Bed • 1 Bath
1,2680,1 Bed • 1 Bath
2,3150,1 Bed • 1 Bath
3,2614,1 Bed • 1 Bath
4,3000,1 Bed • 1 Bath
5,2629,1 Bed • 1 Bath
6,3700,1 Bed • 1 Bath
7,2706,1 Bed • 1 Bath
8,2743,1 Bed • 1 Bath
9,1800,1 Bed • 1 - 2 Baths


## 2 Beds

In [51]:
URL = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/2-beds"
requestURL = requests.get(URL)
content = requestURL.content
soup = BeautifulSoup(content, "html.parser")
rent = [x.text for x in soup.find_all("div", class_="css-uzc059")]
bath = [x.text for x in soup.find_all("div", class_="css-1iq6kk8")]


dict = {'Monthly': rent, 'Number of baths': bath}
dfTwoBed= pd.DataFrame(dict)
dfTwoBed['Monthly'] = dfTwoBed['Monthly'].str.replace('\D+', '')


C:\Users\Jimmy\AppData\Local\Temp/ipykernel_9092/172796148.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  dfTwoBed['Monthly'] = dfTwoBed['Monthly'].str.replace('\D+', '')


In [52]:
dfTwoBed

,Monthly,Number of baths
0,4500,2 Beds • 2 Baths
1,4600,2 Beds • 1 - 2 Baths
2,4100,2 Beds • 1 - 2 Baths
3,265026,2 Beds • 1 Bath
4,4900,2 Beds • 2 Baths
5,3895,2 Beds • 2 Baths
6,4850,2 Beds • 2 Baths
7,3428,2 Beds • 1 Bath
8,3050,2 Beds • 1 Bath
9,3599,2 Beds • 1 Bath


### 3 beds

In [54]:
URL = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/3-beds"
requestURL = requests.get(URL)
content = requestURL.content
soup = BeautifulSoup(content, "html.parser")
rent = [x.text for x in soup.find_all("div", class_="css-uzc059")]
bath = [x.text for x in soup.find_all("div", class_="css-1iq6kk8")]


dict = {'Monthly': rent, 'Number of baths': bath}
dfThreeBed= pd.DataFrame(dict)
dfThreeBed['Monthly'] = dfThreeBed['Monthly'].str.replace('\D+', '')


C:\Users\Jimmy\AppData\Local\Temp/ipykernel_9092/4126310706.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  dfThreeBed['Monthly'] = dfThreeBed['Monthly'].str.replace('\D+', '')


In [55]:
dfThreeBed

,Monthly,Number of baths
0,6550,3 Beds • 2 Baths
1,4500,3 Beds • 2 Baths
2,129,3 Beds • 3 Baths
3,5195,3 Beds • 2 Baths
4,6200,3 Beds • 2 Baths
5,5000,3 Beds • 2 Baths
6,4500,3 Beds • 2 Baths
7,4500,3 Beds • 2 Baths
8,4900,3 Beds • 1 Bath
9,430013,3 Beds • 2 Baths


## Kijiji

In [41]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/bachelor+studio/c37l1700292a27949001"
requestURL2 = requests.get(URL2)
content2 = requestURL2.content

soup2 = BeautifulSoup(content2, "html.parser")

### Studios

In [42]:
realEstate = soup2.findAll("div", class_="container-results large-images")[1]
realEstate2 = [x.find('div', class_="info") for x in realEstate.findAll("div", attrs={"data-listing-id":True})]


In [43]:
realEstateLinks = [x.find('div', class_=["title"]) for x in realEstate2]
realEstateLinks1 = [x.find('a', attrs={"href":True}) for x in realEstate2]
realEstateLinks = ["https://www.kijiji.ca"+ x['href'] for x in realEstateLinks1]

In [44]:
dfKijiji = pd.DataFrame(columns=['Monthly', 'Number of baths'])

for i in range(len(realEstateLinks)):
    requestsIterate = requests.get(realEstateLinks[i])
    soup3 = BeautifulSoup(requestsIterate.text, "html.parser")


    for x in soup3.find_all('div', class_=['realEstateTitle-1440881021']):
        monthlyNum = x.find('span')['content']
        bathroomNum = x.findAll('span', class_="noLabelValue-3861810455")[-1].text

        dfKijiji = dfKijiji.append({'Monthly': monthlyNum, 'Number of baths': bathroomNum}, ignore_index=True)
    

In [45]:
dfKijiji

,Monthly,Number of baths
0,2369.,Bathrooms: 1
1,2650.,Bathrooms: 1
2,1640.,Bathrooms: 1
3,2369.,Bathrooms: 1
4,2195.,Bathrooms: 1
5,2375.,Bathrooms: 1
6,2325.,Bathrooms: 1
7,2900.,Bathrooms: 1
8,2268.,Bathrooms: 1
9,2570.,Bathrooms: 1


### 1 bedroom

In [47]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/1+bedroom/c37l1700292a27949001"
requestURL2 = requests.get(URL2)
content2 = requestURL2.content

soup2 = BeautifulSoup(content2, "html.parser")

realEstate = soup2.findAll("div", class_="container-results large-images")[1]
realEstate2 = [x.find('div', class_="info") for x in realEstate.findAll("div", attrs={"data-listing-id":True})]


dfKijijiOneBed = pd.DataFrame(columns=['Monthly', 'Number of baths'])

for i in range(len(realEstateLinks)):
    requestsIterate = requests.get(realEstateLinks[i])
    soup3 = BeautifulSoup(requestsIterate.text, "html.parser")


    for x in soup3.find_all('div', class_=['realEstateTitle-1440881021']):
        monthlyNum = x.find('span')['content']
        bathroomNum = x.findAll('span', class_="noLabelValue-3861810455")[-1].text

        dfKijijiOneBed = dfKijijiOneBed.append({'Monthly': monthlyNum, 'Number of baths': bathroomNum}, ignore_index=True)
    


In [48]:
dfKijijiOneBed

,Monthly,Number of baths
0,2369.,Bathrooms: 1
1,2650.,Bathrooms: 1
2,1640.,Bathrooms: 1
3,2369.,Bathrooms: 1
4,2195.,Bathrooms: 1
5,2375.,Bathrooms: 1
6,2325.,Bathrooms: 1
7,2900.,Bathrooms: 1
8,2268.,Bathrooms: 1
9,2570.,Bathrooms: 1
